In [49]:
!pip install tensorflow==2.12.0

In [50]:
import os
from google.colab import drive
#drive.mount('/content/drive')

In [51]:
folder_name_Seed = "Seed"

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_name_Seed):
    os.makedirs(folder_name_Seed)

folder_name_Tau = "Tau"

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_name_Tau):
    os.makedirs(folder_name_Tau)

folder_name_MAP = "MAP"

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_name_MAP):
    os.makedirs(folder_name_MAP)

folder_name_MAP = "Data"

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_name_MAP):
    os.makedirs(folder_name_MAP)

folder_name_MAP = "labelData"

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_name_MAP):
    os.makedirs(folder_name_MAP)

In [52]:
!pwd

/content


In [53]:

import networkx as nx
import random
import copy
import pandas as pd
import ast
from ast import literal_eval
from scipy import stats
import numpy as np

import networkx as nx
import pandas as pd

import os

import time
from operator import itemgetter

import math
import numpy as np




# ---------------------------------------------------------------------

def get_wic_list(pathDataset):
    wic = pd.read_csv(pathDataset)
    return dict(zip(np.array(wic['Node'],dtype=str),wic['WIC']))


def nodesRank(rank):
    rWIC = sorted(rank)
    re = []
    for i in rWIC:
        re.append(rank.index(i))
    return re


def compare_tau1(wic,alg_list):
    wic_sort = [i for i,j in sorted(wic.items(),key=lambda x:x[1],reverse=True)]
    wic_sort=[int(i) for i in wic_sort]
    alg_list=[int(i) for i in alg_list]
    tau,_ = stats.kendalltau(nodesRank(alg_list),nodesRank(wic_sort))
    return tau

def rel_at_k(y_true, y_pred, k):
    """ Computes Relevance at k for one sample

    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations

    Returns
    _______a
    score: double
           Relevance at k
    """
    # if y_pred[k] ==  y_true[k]:
    if y_pred[k-1] in  y_true:
        return 1
    else:
        return 0

def precision_at_k(y_true, y_pred, k):
    """ Computes Precision at k for one sample

    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations

    Returns
    _______
    score: double
           Precision at k
    """
    intersection = np.intersect1d(y_true, y_pred[:k])
    return len(intersection) / k


def average_precision_at_k(y_true, y_pred, k):
    """ Computes Average Precision at k for one sample

    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations

    Returns
    _______
    score: double
           Average Precision at k
    """
    ap = 0.0
    for i in range(1, k+1):
        ap += precision_at_k(y_true, y_pred, i) * rel_at_k(y_true, y_pred, i)

    return ap / min(k, len(y_true))



def compare_MAP(wicc,alg_list,per):
    wic_sort = [i for i,j in sorted(wicc.items(),key=lambda x:x[1],reverse=True)]
    k=int(per*len(wic_sort)/100)
    map=average_precision_at_k(wic_sort[:k],alg_list[:k],k)
    return map

def get_algo_list(df_M,dataName,algoName):
    algo_list=[]
    df = df_M.copy()
    df=df[df['Dataset']==dataName]
    df=df[df['Algo']==algoName]
    #algo_list=literal_eval(df['Seed'].iloc[0])
    algo_list=df['Seed'].iloc[0]
    return algo_list


In [55]:
import Base__NDM

def NDM_(G):
    start_time = time.time()
    seedB=Base__NDM.NDM_rank(G)
    seedB=[i for i in seedB]
    timelapse=(time.time() - start_time)
    return seedB,timelapse


def seedAlgo(Algo,G,k  ):
    if Algo=='NDM':
        print('NDM Centrality is selected')
        return NDM_(G)


def loadData(nameDataset,sep=","):
    df = pd.read_csv(nameDataset, sep=sep, names=['FromNodeId', 'ToNodeId'], skiprows=1)
    df['FromNodeId'] = df['FromNodeId'].astype(int)
    df['ToNodeId'] = df['ToNodeId'].astype(int)

    G = nx.from_pandas_edgelist(df, source="FromNodeId", target="ToNodeId")
    G.remove_edges_from(nx.selfloop_edges(G))
    G.remove_nodes_from(['FromNodeId', 'ToNodeId'])
    return G


def files(path):
    filesList=[]
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            filesList.append(file)
    return filesList




input_Datasets_to_pred= [
 'CiaoDVD',
 'Email',
 'Gnutella',
 'GrQc',
 'Hamster',
 'Jazz',
 'LastFM',
 'NetScience',
 'PGP',
 'Politician',
 'PowerGrid',
 'Sex',
]


input_Datasets_to_pred= [
  'Email',
  'PGP',
]



Tau_='Tau_'
seed_='seed_'
MAP_='MAP_'
pathDataset='/content/Data/'
pathLabelDataset='/content/labelData/'

Algorithms=[  'NDM', ]


for Algo in Algorithms:
    df_Seed_result = pd.DataFrame( columns=['Dataset','Algo','Seed','time'])
    df_tau_result = pd.DataFrame( columns=['Dataset','Algo','Tau'])
    df_map_result = pd.DataFrame( columns=['Dataset','Algo','MAP','NumK'])

    for file in input_Datasets_to_pred:

      G =loadData(pathDataset+file+'.csv')
      print('file = ', file)
      k=G.number_of_nodes()

      Seed,timeS=seedAlgo(Algo,G,k )
      Seed=[str(i) for i in Seed]

      print(len(Seed))

      path_WIC_data=pathLabelDataset+file+'_OneNodeIM.csv'
      G_WIC = get_wic_list(path_WIC_data)
      tau=compare_tau1(G_WIC,Seed)
      print(f' {Algo} has tau= {tau} in {file}')

      if G.number_of_nodes()==len(Seed):
          print('Yes,it is equal,  ','-file =', file)
          df2 = {'Dataset': file, 'Algo': Algo, 'Seed': Seed,'time':timeS}
          df_Seed_result=df_Seed_result._append(df2, ignore_index = True)
          df_Seed_result.to_csv(f'/content/Seed/{seed_}'+Algo+'.csv')

          df2= {'Dataset': file,'Algo': Algo, 'Tau': tau}
          df_tau_result=df_tau_result._append(df2, ignore_index = True)
          df_tau_result.to_csv(f'/content/Tau/{Tau_}'+Algo+'.csv')

          print('-------------------------------------------------------------')
          print('done', Algo,' in  ', file)
          print('-------------------------------------------------------------')

          for per in [5,10,15,20]:
            G_WIC=get_wic_list(path_WIC_data)

            MAP=compare_MAP(G_WIC,Seed,per)
            print(file,' MAP=',MAP)

            df2 = {'Dataset': file, 'Algo': Algo, 'MAP': MAP,'numK':per}
            df_map_result=df_map_result._append(df2, ignore_index = True)
            df_map_result.to_csv(f'/content/MAP/{MAP_}'+Algo+'.csv')


file =  Email
NDM Centrality is selected
(1133, 4, 11)
36/36 [==============================] - 1s 3ms/step
1133
 NDM has tau= 0.9582084524964212 in Email
Yes,it is equal,   -file = Email
-------------------------------------------------------------
done NDM  in   Email
-------------------------------------------------------------
Email  MAP= 0.9409366067686004
Email  MAP= 0.9638774180776223
Email  MAP= 0.9692143718617781
Email  MAP= 0.9816752525381208


<ipython-input-55-3bfbc7e86cf8>:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Seed_result=df_Seed_result._append(df2, ignore_index = True)
<ipython-input-55-3bfbc7e86cf8>:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_tau_result=df_tau_result._append(df2, ignore_index = True)
<ipython-input-55-3bfbc7e86cf8>:112: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns w

file =  PGP
NDM Centrality is selected
(10680, 4, 11)
334/334 [==============================] - 1s 2ms/step
10680
 NDM has tau= 0.9212111838383499 in PGP
Yes,it is equal,   -file = PGP
-------------------------------------------------------------
done NDM  in   PGP
-------------------------------------------------------------
PGP  MAP= 0.9669860733272465
PGP  MAP= 0.9610513969561415
PGP  MAP= 0.9687952856379402
PGP  MAP= 0.9778586988892313


In [ ]:
import shutil
# Zip the folder
# Download the ZIP file
from google.colab import files
shutil.make_archive(folder_name_Tau, 'zip', folder_name_Tau)
zip_file_path = folder_name_Tau + ".zip"
files.download(zip_file_path)



# Zip the folder
shutil.make_archive(folder_name_MAP, 'zip', folder_name_MAP)
zip_file_path = folder_name_MAP + ".zip"
files.download(zip_file_path)


# Zip the folder
shutil.make_archive(folder_name_Seed, 'zip', folder_name_Seed)
zip_file_path = folder_name_Seed + ".zip"
files.download(zip_file_path)